In [1]:
import argparse
import datetime
import logging
import math
import pickle
import re
from collections import Counter, defaultdict

import pandas as pd
import torch
import tqdm

from pyrocov.geo import get_canonical_location_generator, gisaid_normalize
from pyrocov.mutrans import START_DATE
from pyrocov.sarscov2 import nuc_mutations_to_aa_mutations
from pyrocov.usher import (
    FineToMeso,
    load_mutation_tree,
    load_proto,
    prune_mutation_tree,
    refine_mutation_tree,
)
from pyrocov.util import gzip_open_tqdm

logger = logging.getLogger(__name__)
logging.basicConfig(format="%(relativeCreated) 9d %(message)s", level=logging.INFO)

DATE_FORMATS = {7: "%Y-%m", 10: "%Y-%m-%d"}

In [8]:
# Copied from scripts/preprocess_usher.py
# DATE_FORMATS = {7: "%Y-%m", 10: "%Y-%m-%d"}

# def try_parse_date(string):
#     fmt = DATE_FORMATS.get(len(string))
#     if fmt is not None:
#         return datetime.datetime.strptime(string, fmt)
    
DATE_FORMATS = {7: "%Y-%m", 10: "%Y-%m-%d"}

def try_parse_date(string):
    fmt = DATE_FORMATS.get(len(string))
    if fmt is not None:
        try:
            return datetime.datetime.strptime(string, fmt)
        except ValueError:
            return    
    
    

parser = argparse.ArgumentParser(description="Preprocess pangolin mutations")
parser.add_argument(
    "--usher-metadata-file-in", default="results/usher/metadata.tsv"
)
parser.add_argument(
    "--nextstrain-metadata-file-in", default="results/nextstrain/metadata.tsv"
)
parser.add_argument("--gisaid-metadata-file-in", default="")
parser.add_argument("--tree-file-in", default="results/usher/all.masked.pb")
parser.add_argument("--tree-file-out", default="results/lineageTree.fine.pb")
parser.add_argument("--stats-file-out", default="results/stats.pkl")
parser.add_argument("--recover-missing-usa-state", action="store_true")
parser.add_argument("-s", "--max-skippage", type=float, default=1e7)
parser.add_argument("-c", "--max-num-clades", default="2000,3000,5000,10000")
parser.add_argument("--start-date", default=START_DATE)

# args = parser.parse_args()
args = parser.parse_args(args=[])

args.start_date = try_parse_date(args.start_date)

In [9]:
import gzip

from pyrocov.usher import load_proto
from pyrocov.usher import parsimony_pb2

In [12]:
# filename = "results/gisaid/gisaidAndPublic.2021-03-26.masked.pb.gz"
filename = "results/gisaid/gisaidAndPublic.2022-09-14.masked.pb.gz"
proto, tree = load_proto(filename)

In [13]:
clades = list(tree.find_clades())

In [14]:
print(len(proto.node_mutations))
print(len(proto.metadata))
print(len(clades))

8248803
8248803
8248803


In [15]:
clades[:10]

[Clade(branch_length=0.0),
 Clade(branch_length=1.0, name='CHN/YN-0306-466/2020|MT396241.1|2020-03-06'),
 Clade(branch_length=1.0, name='DP0803|LC571037.1|2020-02-17'),
 Clade(branch_length=1.0),
 Clade(branch_length=2.0, name='England/LEED-2A8B52/2020|OA971832.1|2020-04-04'),
 Clade(branch_length=1.0, name='England/SHEF-C06CE/2020|2020-03-25'),
 Clade(branch_length=1.0),
 Clade(branch_length=1.0),
 Clade(branch_length=0.0, name='England/SHEF-C07F8/2020|2020-03-21'),
 Clade(branch_length=1.0, name='England/SHEF-C0145/2020|2020-03-25')]

In [16]:
open_ = gzip.open if filename.endswith(".gz") else open
with open_(filename, "rb") as f:
    proto = parsimony_pb2.data.FromString(f.read())  # type: ignore

In [17]:
mutations=list(proto.node_mutations)
metadata = list(proto.metadata)
type(metadata[0])

parsimony_pb2.node_metadata

In [18]:
len(clades)

8248803